In [1]:
from __future__ import print_function
from pathlib import Path
from collections import defaultdict
import functools as fn
import math
import statistics as stat
import sys, os
import pandas as pd
import numpy as np

import nilmtk
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../')

import adinilm.functions as F
from adinilm.utils import paths_manager as pathsman

In [28]:
rcParams['figure.figsize'] = (17, 6)
start_time = "2015-01-01"
end_time = "2015-03-30"

In [29]:
train_dataset = nilmtk.DataSet(pathsman.UKDALE_H5_PATH)
train_dataset.set_window(start=start_time, end=end_time)

In [30]:
BUILDING = 1
FRIDGE_FREEZER = "fridge freezer"
KETTLE = "kettle"
DISHWASHER = "dish washer"
MICROWAVE = "microwave"
WASHER_DRYER = "washer dryer"

applist = [FRIDGE_FREEZER, KETTLE, DISHWASHER, MICROWAVE, WASHER_DRYER]

In [31]:
train_elec = train_dataset.buildings[BUILDING].elec

In [32]:
print(f"Activation stats in the period [{start_time}, {end_time}]")
row = []

for app in applist:
    col = []
    app_name = app.upper()
    activations = train_elec[app].get_activations()
    total_num = len(activations)
    average_length = stat.mean(map(len, activations))
    max_length = max(map(len, activations))
    min_length = min(map(len, activations))
    col.append(total_num)
    col.append(average_length)
    col.append(max_length)
    col.append(min_length)
    row.append(col)

pd.DataFrame(row, columns=["No. of Activations", "Average Length", "Max Length", "Min Length"], index=applist)

Activation stats in the period [2015-01-01, 2015-03-30]


,No. of Activations,Average Length,Max Length,Min Length
fridge freezer,2092,249.163958,3570,53
kettle,458,27.174672,103,7
dish washer,31,1086.935484,1181,598
microwave,804,18.865672,142,7
washer dryer,98,1008.918367,1308,246


In [33]:
def visualize_activation(appliance, num_acts=None, ylim=None):
        activations = train_elec[appliance].get_activations()

        num_acts = num_acts or 1

        for activation in activations[:num_acts]:
                activation.plot()
                plt.ylim(*(ylim or (-10, 2500))) 

In [34]:
def collect_sequence(appliance, num_windows, seq_len=100, mode="adaptive"):
        collections = []
        for activation in train_elec[appliance].get_activations():
                collections = collections + F.scan_sequences(np.array(activation), seq_len=seq_len, num_windows=num_windows, mode=mode)
        return collections

In [41]:
info = defaultdict(dict)
windowlist = [1, 5, 70, 3, 23]

for app, window in zip(applist, windowlist):
        collected_sequences = collect_sequence(app, num_windows=window)
        info[app]["Total collected sequences"] = len(collected_sequences)

pd.DataFrame(info)

,fridge freezer,kettle,dish washer,microwave,washer dryer
Total collected sequences,2092,2286,2160,2404,2150
